In [ ]:
import os
import re
import yaml
import numpy as np
import rasterio
from glob import glob
from tqdm import tqdm
from joblib import Parallel, delayed
from rasterio.warp import reproject, Resampling

# **NDVI and Cropmap-Based Cropland Analysis**

This notebook performs a comprehensive analysis of croplands using NDVI and Cropmap data.  
The key tasks include:

1. **Monthly NDVI Maximum and NDVI Range Calculation**
2. **Pure Cropland-Based Monthly Mean and Standard Deviation Calculation**
3. **TANDVI and TANDVI Range Calculation**
4. **Cropmap Resampling**
5. **MedianCD Calculation for Cropland Pixels**
6. **Parallel Processing for Multi-Region and Multi-Year NDVI Data**

In [2]:
def extract_doy_from_filename(filename):
    """파일 이름에서 DOY(Day of Year)를 추출하는 함수."""
    doy_pattern = re.compile(r"_(\d{3})\.tif$")
    match = doy_pattern.search(filename)
    if match:
        return int(match.group(1))
    return None

def get_month_from_doy(doy):
    """DOY(Day of Year)를 월로 변환하는 함수."""
    if doy <= 31:  # January
        return 1
    elif doy <= 59:  # February
        return 2
    elif doy <= 90:  # March
        return 3
    elif doy <= 120:  # April
        return 4
    elif doy <= 151:  # May
        return 5
    elif doy <= 181:  # June
        return 6
    elif doy <= 212:  # July
        return 7
    elif doy <= 243:  # August
        return 8
    elif doy <= 273:  # September
        return 9
    elif doy <= 304:  # October
        return 10
    elif doy <= 334:  # November
        return 11
    else:  # December
        return 12

## **1. Monthly NDVI Maximum and NDVI Range Calculation**

This section defines functions for calculating the **monthly maximum NDVI** and **monthly NDVI range** (difference between maximum and minimum NDVI) from daily NDVI files. These results are saved as 12-band TIFF files, where each band represents a specific month.

In [ ]:
def calculate_monthly_max_ndvi(ndvi_files, output_tif):
    monthly_ndvi = {month: [] for month in range(1, 13)}
    for ndvi_file in ndvi_files:
        doy = extract_doy_from_filename(ndvi_file)
        if doy is not None:
            month = get_month_from_doy(doy)
            monthly_ndvi[month].append(ndvi_file)

    with rasterio.open(ndvi_files[0]) as src:
        profile = src.profile
        height, width = src.shape

    monthly_max_ndvi = []
    for month in range(1, 13):
        if monthly_ndvi[month]:
            max_ndvi = np.full((height, width), np.iinfo(np.int16).min, dtype=np.int16)
            for ndvi_file in monthly_ndvi[month]:
                with rasterio.open(ndvi_file) as src:
                    ndvi = src.read(1).astype(np.int16)
                    max_ndvi = np.maximum(max_ndvi, ndvi)
            monthly_max_ndvi.append(max_ndvi)
        else:
            monthly_max_ndvi.append(np.zeros((height, width), dtype=np.int16))

    profile.update(dtype=rasterio.int16, count=12)
    with rasterio.open(output_tif, 'w', **profile) as dst:
        for i, max_ndvi in enumerate(monthly_max_ndvi, start=1):
            dst.write(max_ndvi, i)

    print(f"월별 최대 NDVI 값이 {output_tif}에 Int16 형식으로 저장되었습니다.")

def calculate_monthly_ndvi_range(ndvi_files, output_range_tif):
    monthly_ndvi = {month: [] for month in range(1, 13)}
    for ndvi_file in ndvi_files:
        doy = extract_doy_from_filename(ndvi_file)
        if doy is not None:
            month = get_month_from_doy(doy)
            monthly_ndvi[month].append(ndvi_file)

    with rasterio.open(ndvi_files[0]) as src:
        profile = src.profile
        height, width = src.shape

    monthly_ndvi_range = []
    for month in range(1, 13):
        if monthly_ndvi[month]:
            max_ndvi = np.full((height, width), np.iinfo(np.int16).min, dtype=np.int16)
            min_ndvi = np.full((height, width), np.iinfo(np.int16).max, dtype=np.int16)
            for ndvi_file in monthly_ndvi[month]:
                with rasterio.open(ndvi_file) as src:
                    ndvi = src.read(1).astype(np.int16)
                    max_ndvi = np.maximum(max_ndvi, ndvi)
                    min_ndvi = np.minimum(min_ndvi, ndvi)
            ndvi_range = max_ndvi - min_ndvi
            monthly_ndvi_range.append(ndvi_range)
        else:
            monthly_ndvi_range.append(np.zeros((height, width), dtype=np.int16))

    profile.update(dtype=rasterio.int16, count=12)
    with rasterio.open(output_range_tif, 'w', **profile) as dst:
        for i, ndvi_range in enumerate(monthly_ndvi_range, start=1):
            dst.write(ndvi_range, i)

    print(f"월별 NDVI range 값이 {output_range_tif}에 Int16 형식으로 저장되었습니다.")

def process_year(region, year, ndvi_path, output_max_tif, output_range_tif):
    ndvi_files_year = glob(os.path.join(ndvi_path, f"NDVI_*_{year}_*.tif"))
    if ndvi_files_year:
        calculate_monthly_max_ndvi(ndvi_files_year, output_max_tif)
        calculate_monthly_ndvi_range(ndvi_files_year, output_range_tif)
    else:
        print(f"{region} 지역의 {year}년도의 NDVI 파일이 존재하지 않습니다.")

def process_all_years_for_region_in_parallel(region, start_year, end_year, base_ndvi_path, output_max_dir, output_range_dir):
    os.makedirs(output_max_dir, exist_ok=True)
    os.makedirs(output_range_dir, exist_ok=True)

    Parallel(n_jobs=-1)(delayed(process_year)(
        region,
        year,
        os.path.join(base_ndvi_path, str(year)),
        os.path.join(output_max_dir, f"Monthly_Max_NDVI_{year}.tif"),
        os.path.join(output_range_dir, f"Monthly_NDVI_Range_{year}.tif")
    ) for year in range(start_year, end_year + 1))

def process_all_regions(start_year, end_year, base_preprocessed_path):
    regions = [d for d in os.listdir(base_preprocessed_path) if os.path.isdir(os.path.join(base_preprocessed_path, d))]
    Parallel(n_jobs=5)(delayed(process_all_years_for_region_in_parallel)(
        region,
        start_year,
        end_year,
        os.path.join(base_preprocessed_path, region, 'preprocessed'),
        os.path.join(base_preprocessed_path, region, 'monthly_max'),
        os.path.join(base_preprocessed_path, region, 'range')
    ) for region in regions)

# 설정
base_preprocessed_path = "/mnt/raid5/1114/preprocessed/"
start_year = 2013
end_year = 2023

# 모든 지역에 대해 NDVI 데이터 병렬 처리
process_all_regions(start_year, end_year, base_preprocessed_path)

## **2. Pure Cropland-Based Statistics and TANDVI Calculation**

This section defines functions to compute **pure cropland-based statistics** and **TANDVI (Temporal Anomaly NDVI)** for the given regions and years.  
The workflow involves calculating the **monthly mean** and **standard deviation** for pure cropland pixels, and using these statistics to compute **TANDVI** and **TANDVI Range**.

In [ ]:
def calculate_pure_crop_mean_and_std(ndvi_files, output_mean_tif, output_std_tif, description):
    if not ndvi_files:
        print(f"{description} 파일이 존재하지 않아 평균 및 표준편차를 계산할 수 없습니다.")
        return

    with rasterio.open(ndvi_files[0]) as src:
        profile = src.profile
        height, width = src.shape
        profile.update(count=12, dtype=rasterio.float32)

    mean_data = np.zeros((12, height, width), dtype=np.float32)
    std_data = np.zeros((12, height, width), dtype=np.float32)

    for band in range(1, 13):
        band_values = []
        for ndvi_file in ndvi_files:
            with rasterio.open(ndvi_file) as src:
                data = src.read(band).astype(np.float32)
                band_values.append(data)

        band_values = np.array(band_values)
        median_value = np.median(band_values, axis=0)
        pure_crop_values = np.where(band_values >= median_value, band_values, np.nan)

        mean_data[band - 1] = np.nanmean(pure_crop_values, axis=0)
        std_data[band - 1] = np.nanstd(pure_crop_values, axis=0)

    with rasterio.open(output_mean_tif, 'w', **profile) as dst_mean, \
         rasterio.open(output_std_tif, 'w', **profile) as dst_std:
        for band in range(1, 13):
            dst_mean.write(mean_data[band - 1].astype(np.float32), band)
            dst_std.write(std_data[band - 1].astype(np.float32), band)

    print(f"순수 작물 기반 월별 평균 {description} 값이 {output_mean_tif}에 저장되었습니다.")
    print(f"순수 작물 기반 월별 표준편차 {description} 값이 {output_std_tif}에 저장되었습니다.")

def calculate_pure_crop_tandvi_and_tandvirange(ndvi_file, range_file, mean_ndvi_file, std_ndvi_file, mean_range_file, std_range_file, output_tandvi_tif, output_tandvirange_tif):
    with rasterio.open(ndvi_file) as src_ndvi, \
         rasterio.open(range_file) as src_range, \
         rasterio.open(mean_ndvi_file) as src_mean_ndvi, \
         rasterio.open(std_ndvi_file) as src_std_ndvi, \
         rasterio.open(mean_range_file) as src_mean_range, \
         rasterio.open(std_range_file) as src_std_range:

        profile = src_ndvi.profile
        profile.update(count=12, dtype=rasterio.float32)

        with rasterio.open(output_tandvi_tif, 'w', **profile) as dst_tandvi, \
             rasterio.open(output_tandvirange_tif, 'w', **profile) as dst_tandvirange:
            for band in range(1, 13):
                ndvi_month = src_ndvi.read(band).astype(np.float32)
                ndvi_range_month = src_range.read(band).astype(np.float32)

                mean_ndvi = src_mean_ndvi.read(band).astype(np.float32)
                std_ndvi = src_std_ndvi.read(band).astype(np.float32)
                mean_range = src_mean_range.read(band).astype(np.float32)
                std_range = src_std_range.read(band).astype(np.float32)

                epsilon = 1e-6

                tandvi = (ndvi_month - mean_ndvi) / (std_ndvi + epsilon)
                tandvirange = (ndvi_range_month - mean_range) / (std_range + epsilon)

                dst_tandvi.write(tandvi.astype(np.float32), band)
                dst_tandvirange.write(tandvirange.astype(np.float32), band)

        print(f"TANDVI 값이 {output_tandvi_tif}에 저장되었습니다.")
        print(f"TANDVIrange 값이 {output_tandvirange_tif}에 저장되었습니다.")

def process_pure_crop_stats(pure_crop_start_year, pure_crop_end_year, region_dir):
    output_mean_ndvi_tif = os.path.join(region_dir, f"Pure_Crop_Mean_NDVI_{pure_crop_start_year}_{pure_crop_end_year}.tif")
    output_std_ndvi_tif = os.path.join(region_dir, f"Pure_Crop_STD_NDVI_{pure_crop_start_year}_{pure_crop_end_year}.tif")
    output_mean_range_tif = os.path.join(region_dir, f"Pure_Crop_Mean_NDVI_Range_{pure_crop_start_year}_{pure_crop_end_year}.tif")
    output_std_range_tif = os.path.join(region_dir, f"Pure_Crop_STD_NDVI_Range_{pure_crop_start_year}_{pure_crop_end_year}.tif")

    ndvi_files = glob(os.path.join(region_dir, "monthly_max", "*.tif"))
    range_files = glob(os.path.join(region_dir, "range", "*.tif"))

    if not (os.path.exists(output_mean_ndvi_tif) and os.path.exists(output_std_ndvi_tif)):
        calculate_pure_crop_mean_and_std(ndvi_files, output_mean_ndvi_tif, output_std_ndvi_tif, "NDVI")

    if not (os.path.exists(output_mean_range_tif) and os.path.exists(output_std_range_tif)):
        calculate_pure_crop_mean_and_std(range_files, output_mean_range_tif, output_std_range_tif, "NDVI Range")

    return output_mean_ndvi_tif, output_std_ndvi_tif, output_mean_range_tif, output_std_range_tif

def process_year(year, region_dir, mean_ndvi_tif, std_ndvi_tif, mean_range_tif, std_range_tif):
    ndvi_month_file = os.path.join(region_dir, "monthly_max", f"Monthly_Max_NDVI_{year}.tif")
    range_month_file = os.path.join(region_dir, "range", f"Monthly_NDVI_Range_{year}.tif")
    output_tandvi_tif = os.path.join(region_dir, "tandvi_", f"TANDVI_{year}.tif")
    output_tandvirange_tif = os.path.join(region_dir, "tandvirange_", f"TANDVIrange_{year}.tif")

    os.makedirs(os.path.join(region_dir, "tandvi_"), exist_ok=True)
    os.makedirs(os.path.join(region_dir, "tandvirange_"), exist_ok=True)

    if os.path.exists(ndvi_month_file) and os.path.exists(range_month_file):
        calculate_pure_crop_tandvi_and_tandvirange(
            ndvi_month_file, range_month_file,
            mean_ndvi_tif, std_ndvi_tif,
            mean_range_tif, std_range_tif,
            output_tandvi_tif, output_tandvirange_tif
        )
    else:
        print(f"{year}년도에 필요한 NDVI 또는 NDVI Range 파일이 {region_dir}에 존재하지 않습니다.")

def process_all_years_parallel(start_year, end_year, region_dir, mean_ndvi_tif, std_ndvi_tif, mean_range_tif, std_range_tif):
    years = list(range(start_year, end_year + 1))
    Parallel(n_jobs=-1)(
        delayed(process_year)(
            year, region_dir, mean_ndvi_tif, std_ndvi_tif, mean_range_tif, std_range_tif
        )
        for year in tqdm(years, desc=f"Processing years for region {os.path.basename(region_dir)}")
    )

def process_all_regions(base_dir, pure_crop_start_year, pure_crop_end_year, tandvi_start_year, tandvi_end_year, target_regions):
    Parallel(n_jobs=-1)(
        delayed(process_region)(
            base_dir, region, pure_crop_start_year, pure_crop_end_year, tandvi_start_year, tandvi_end_year
        )
        for region in target_regions
    )

def process_region(base_dir, region, pure_crop_start_year, pure_crop_end_year, tandvi_start_year, tandvi_end_year):
    region_dir = os.path.join(base_dir, region)

    if os.path.isdir(os.path.join(region_dir, "preprocessed")):
        print(f"Processing region: {region}")
        mean_ndvi_tif, std_ndvi_tif, mean_range_tif, std_range_tif = process_pure_crop_stats(
            pure_crop_start_year, pure_crop_end_year, region_dir
        )
        process_all_years_parallel(
            tandvi_start_year, tandvi_end_year, region_dir,
            mean_ndvi_tif, std_ndvi_tif, mean_range_tif, std_range_tif
        )
    else:
        print(f"{region} 지역에 'preprocessed' 폴더가 존재하지 않습니다.")

# 설정
base_dir = "/mnt/raid5/1114/preprocessed/"

# Pure Crop 계산에 사용할 기간
pure_crop_start_year = 2013
pure_crop_end_year = 2019

# TANDVI 및 TANDVIrange를 계산할 기간
tandvi_start_year = 2020
tandvi_end_year = 2023

# 대상 지역명 리스트 설정
target_regions = [folder_name for folder_name in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, folder_name))]

# 지정된 지역에 대해 NDVI 데이터 병렬 처리
process_all_regions(base_dir, pure_crop_start_year, pure_crop_end_year, tandvi_start_year, tandvi_end_year, target_regions)

## **3. NDVI and Cropmap Resampling with MedianCD Calculation**

This notebook processes **NDVI** and **Cropmap** data to calculate the **MedianCD (Median Cropland Deviation)** for each region by resampling the cropmap and extracting cropland pixel values.

In [ ]:
# 경로 설정
ndvi_base_dir = "/mnt/raid5/1114/preprocessed/"
cropmap_base_dir = "/mnt/raid5/1114/cropmap/"
output_base_dir = "/mnt/raid5/1114/cropmap/"

# 지역별 cropmap 리샘플링 함수
def resample_cropmaps(ndvi_base_dir, cropmap_base_dir, output_base_dir):
    # 지역 폴더를 탐색하여 지역명을 추출
    region_names = [name for name in os.listdir(ndvi_base_dir) if os.path.isdir(os.path.join(ndvi_base_dir, name))]
    
    # 각 지역별로 cropmap 리샘플링
    for region in region_names:
        # 지역의 가장 최신 연도의 NDVI 파일을 참조 파일로 사용
        ndvi_path = os.path.join(ndvi_base_dir, region, "preprocessed/2023", f"NDVI_{region}_2023_361.tif")
        cropmap_path = os.path.join(cropmap_base_dir, f"cropmap_{region}.tif")
        output_path = os.path.join(output_base_dir, f"resampled_cropmap_{region}.tif")
        
        # NDVI 참조 파일과 cropmap 파일을 기준으로 리샘플링 수행
        if os.path.exists(ndvi_path) and os.path.exists(cropmap_path):
            with rasterio.open(ndvi_path) as ref_src:
                ref_crs = ref_src.crs
                ref_transform = ref_src.transform
                ref_width = ref_src.width
                ref_height = ref_src.height
                
                out_meta = ref_src.meta.copy()
                out_meta.update({
                    "crs": ref_crs,
                    "transform": ref_transform,
                    "width": ref_width,
                    "height": ref_height
                })
                
                # cropmap 파일 열기
                with rasterio.open(cropmap_path) as crop_src:
                    # 출력 파일에 리샘플링된 데이터 저장
                    with rasterio.open(output_path, 'w', **out_meta) as dst:
                        for i in range(1, crop_src.count + 1):
                            reproject(
                                source=rasterio.band(crop_src, i),
                                destination=rasterio.band(dst, i),
                                src_transform=crop_src.transform,
                                src_crs=crop_src.crs,
                                dst_transform=ref_transform,
                                dst_crs=ref_crs,
                                resampling=Resampling.nearest
                            )
            print(f"Resampled cropmap created for {region}: {output_path}")
        else:
            print(f"NDVI or Cropmap file missing for {region}")

# 함수 실행
resample_cropmaps(ndvi_base_dir, cropmap_base_dir, output_base_dir)

In [ ]:
# Cropmap에서 경작지를 나타내는 픽셀 값
CROPLAND_VALUE = 255

# cropmap 파일에서 경작지 픽셀 마스크 생성 함수
def get_cropland_mask(cropmap_file):
    with rasterio.open(cropmap_file) as src:
        crop_data = src.read(1)  # 첫 번째 밴드 읽기
        # 경작지 픽셀을 나타내는 마스크 생성
        cropland_mask = (crop_data == CROPLAND_VALUE)
    return cropland_mask

# NDVI 또는 NDVI Range 파일에서 경작지 픽셀 값 추출 및 MedianCD 계산 함수
def calculate_median_cd(ndvi_files, cropland_mask, output_median_cd_yaml, data_type="NDVI"):
    median_cd_values = {}
    
    for month in range(1, 13):
        monthly_values = []
        for ndvi_file in tqdm(ndvi_files, desc=f"{data_type} - Month {month}", leave=False):
            with rasterio.open(ndvi_file) as src_ndvi:
                if month <= src_ndvi.count:  # 파일에 해당 월 데이터가 존재하는지 확인
                    ndvi_data = src_ndvi.read(month)
                    # 경작지 마스크에 해당하는 NDVI 값만 필터링
                    data_cropped = ndvi_data[cropland_mask]
                    if data_cropped.size > 0:
                        monthly_values.extend(data_cropped)
        
        # 월별 중앙값 계산
        if monthly_values:  # 데이터가 있는 경우에만 중앙값 계산
            median_cd_values[f"Month_{month}"] = float(np.median(monthly_values))
        else:
            median_cd_values[f"Month_{month}"] = None  # 데이터가 없을 경우 None 할당

    # YAML 파일로 결과 저장
    with open(output_median_cd_yaml, 'w') as yaml_file:
        yaml.dump(median_cd_values, yaml_file)

    print(f"{data_type} MedianCD 파일이 생성되었습니다: {output_median_cd_yaml}")

# 통합 프로세스 함수
def process_all(region_name, ndvi_dir, ndvi_range_dir, cropmap_base_path, output_dir):
    # 1. Cropmap 파일에서 경작지 픽셀 마스크 가져오기
    cropmap_file = os.path.join(cropmap_base_path, f"resampled_cropmap_{region_name}.tif")
    if not os.path.exists(cropmap_file):
        print(f"Skipping {region_name}: cropmap file not found.")
        return
    cropland_mask = get_cropland_mask(cropmap_file)

    # 2013~2023년의 Monthly Max NDVI 파일을 선택
    ndvi_files = sorted(glob(os.path.join(ndvi_dir, "Monthly_Max_NDVI_20[1-2][0-9].tif")))
    output_median_cd_yaml_ndvi = os.path.join(output_dir, "MedianCD_Monthly.yaml")
    calculate_median_cd(ndvi_files, cropland_mask, output_median_cd_yaml_ndvi, data_type="NDVI")

    # 2013~2023년의 Monthly NDVI Range 파일을 선택
    ndvi_range_files = sorted(glob(os.path.join(ndvi_range_dir, "Monthly_NDVI_Range_20[1-2][0-9].tif")))
    output_median_cd_yaml_range = os.path.join(output_dir, "MedianCD_Range_Monthly.yaml")
    calculate_median_cd(ndvi_range_files, cropland_mask, output_median_cd_yaml_range, data_type="NDVI Range")

# 병렬 처리할 지역별 작업 설정 함수
def process_region(region_name, base_dir, cropmap_base_path):
    ndvi_dir = os.path.join(base_dir, f"preprocessed/{region_name}/monthly_max")
    ndvi_range_dir = os.path.join(base_dir, f"preprocessed/{region_name}/range")
    output_dir = os.path.join(base_dir, f"preprocessed/{region_name}")  # 결과 저장 경로
    process_all(region_name, ndvi_dir, ndvi_range_dir, cropmap_base_path, output_dir)

# 파일 경로 설정 및 실행
def main():
    base_dir = "/mnt/raid5/1114/"
    cropmap_base_path = os.path.join(base_dir, "cropmap")
    preprocessed_base_dir = os.path.join(base_dir, "preprocessed")

    # preprocessed 디렉토리 하위 폴더명으로 지역 리스트 생성
    region_names = [name for name in os.listdir(preprocessed_base_dir) if os.path.isdir(os.path.join(preprocessed_base_dir, name))]

    # 병렬로 각 지역에 대해 통합 프로세스 실행
    Parallel(n_jobs=-1)(
        delayed(process_region)(region_name, base_dir, cropmap_base_path) 
        for region_name in tqdm(region_names, desc="Processing Regions")
    )

if __name__ == "__main__":
    main()

# **FALLOW Detection using FANTA Algorithm**

This notebook implements the **Fallow-land Algorithm based on Neighborhood and Temporal Anomalies (FANTA)** as described in the following paper:

**Reference**:  
Wallace, C. S., Thenkabail, P., Rodriguez, J. R., & Brown, M. K. (2017).  
Fallow-land Algorithm based on Neighborhood and Temporal Anomalies (FANTA) to map planted versus fallowed croplands using MODIS data to assist in drought studies leading to water and food security assessments.  
*GIScience & Remote Sensing, 54(2), 258-282.*

The FANTA algorithm is designed to identify **fallowed croplands** based on temporal NDVI anomalies and neighborhood characteristics, aiding in drought analysis, water management, and food security assessments. This implementation adapts the algorithm for higher-resolution satellite data (e.g., Landsat or Sentinel) and calculates fallow areas using **TANDVI**, **TANDVI Range**, **NDVI**, and **NDVI Range** metrics.

---

## **Overview of the FANTA Algorithm**

The FANTA algorithm identifies fallow croplands based on the following key components:

1. **Temporal NDVI Anomalies**:  
   The algorithm detects croplands that exhibit NDVI values significantly lower than the expected median NDVI during key growing months.
   
2. **Neighborhood Anomalies**:  
   By analyzing the spatial neighborhood of each pixel, the algorithm ensures that detected fallow croplands are not isolated but occur in clusters, representing true fallow conditions.

3. **Key Growing Months**:  
   The algorithm focuses on critical growing months (April, May, June, July) to assess anomalies in crop growth.

---

In [ ]:
# Q1: FALLOW detection based on TANDVI
def calculate_fallow_tandvi(tandvi_dir, year, output_fallow_tif):
    tandvi_file = os.path.join(tandvi_dir, f"TANDVI_{year}.tif")
    if not os.path.exists(tandvi_file):
        print(f"{tandvi_file} does not exist. Skipping.")
        return
    with rasterio.open(tandvi_file) as src:
        profile = src.profile
        height, width = src.shape
        profile.update(count=1, dtype=rasterio.uint8)
        tandvi_april = src.read(4).astype(np.float32)
        tandvi_may = src.read(5).astype(np.float32)
        tandvi_june = src.read(6).astype(np.float32)
        tandvi_july = src.read(7).astype(np.float32)
        fallow = np.zeros((height, width), dtype=np.uint8)
        condition1 = (tandvi_may < -3) & (tandvi_june < -3) & (tandvi_july < -3)
        condition2 = (tandvi_april < -3) & (tandvi_may < -3) & (tandvi_june < -3)
        fallow[condition1 | condition2] = 255
        with rasterio.open(output_fallow_tif, 'w', **profile) as dst:
            dst.write(fallow, 1)
    print(f"FALLOW Q1 saved: {output_fallow_tif}")

# Q2: FALLOW detection based on TANDVI Range
def calculate_fallow_tandvirange(tandvirange_dir, year, output_fallow_tif):
    tandvirange_file = os.path.join(tandvirange_dir, f"TANDVIrange_{year}.tif")
    if not os.path.exists(tandvirange_file):
        print(f"{tandvirange_file} does not exist. Skipping.")
        return
    with rasterio.open(tandvirange_file) as src:
        profile = src.profile
        height, width = src.shape
        profile.update(count=1, dtype=rasterio.uint8)
        tandvirange_april = src.read(4).astype(np.float32)
        tandvirange_may = src.read(5).astype(np.float32)
        tandvirange_june = src.read(6).astype(np.float32)
        tandvirange_july = src.read(7).astype(np.float32)
        fallow = np.zeros((height, width), dtype=np.uint8)
        condition1 = (tandvirange_may < -3) & (tandvirange_june < -3) & (tandvirange_july < -3)
        condition2 = (tandvirange_april < -3) & (tandvirange_may < -3) & (tandvirange_june < -3)
        fallow[condition1 | condition2] = 255
        with rasterio.open(output_fallow_tif, 'w', **profile) as dst:
            dst.write(fallow, 1)
    print(f"FALLOW Q2 saved: {output_fallow_tif}")

# Q3: FALLOW detection based on NDVI
def calculate_fallow_ndvi(ndvi_dir, median_cd_file, year, output_fallow_tif):
    ndvi_file = os.path.join(ndvi_dir, f"Monthly_Max_NDVI_{year}.tif")
    if not os.path.exists(ndvi_file):
        print(f"{ndvi_file} does not exist. Skipping.")
        return
    with open(median_cd_file, 'r') as yaml_file:
        median_cd_values = yaml.safe_load(yaml_file)
    with rasterio.open(ndvi_file) as src:
        profile = src.profile
        height, width = src.shape
        profile.update(count=1, dtype=rasterio.uint8)
        ndvi_april = src.read(4).astype(np.float32)
        ndvi_may = src.read(5).astype(np.float32)
        ndvi_june = src.read(6).astype(np.float32)
        ndvi_july = src.read(7).astype(np.float32)
        median_cd_max = max(median_cd_values.get("Month_4", 0), median_cd_values.get("Month_5", 0),
                            median_cd_values.get("Month_6", 0), median_cd_values.get("Month_7", 0))
        ndvi_max = np.maximum(np.maximum(ndvi_april, ndvi_may), np.maximum(ndvi_june, ndvi_july))
        fallow = np.zeros((height, width), dtype=np.uint8)
        fallow[ndvi_max < 0.8 * median_cd_max] = 255
        with rasterio.open(output_fallow_tif, 'w', **profile) as dst:
            dst.write(fallow, 1)
    print(f"FALLOW Q3 saved: {output_fallow_tif}")

# Q4: FALLOW detection based on NDVI Range
def calculate_fallow_ndvi_range(ndvi_range_dir, median_cd_file, year, output_fallow_tif):
    ndvi_range_file = os.path.join(ndvi_range_dir, f"Monthly_NDVI_Range_{year}.tif")
    if not os.path.exists(ndvi_range_file):
        print(f"{ndvi_range_file} does not exist. Skipping.")
        return
    with open(median_cd_file, 'r') as yaml_file:
        median_cd_values = yaml.safe_load(yaml_file)
    with rasterio.open(ndvi_range_file) as src:
        profile = src.profile
        height, width = src.shape
        profile.update(count=1, dtype=rasterio.uint8)
        ndvi_april = src.read(4).astype(np.float32)
        ndvi_may = src.read(5).astype(np.float32)
        ndvi_june = src.read(6).astype(np.float32)
        ndvi_july = src.read(7).astype(np.float32)
        median_cd_max = max(median_cd_values.get("Month_4", 0), median_cd_values.get("Month_5", 0),
                            median_cd_values.get("Month_6", 0), median_cd_values.get("Month_7", 0))
        ndvi_range_max = np.maximum(np.maximum(ndvi_april, ndvi_may), np.maximum(ndvi_june, ndvi_july))
        fallow = np.zeros((height, width), dtype=np.uint8)
        fallow[ndvi_range_max < 0.8 * median_cd_max] = 255
        with rasterio.open(output_fallow_tif, 'w', **profile) as dst:
            dst.write(fallow, 1)
    print(f"FALLOW Q4 saved: {output_fallow_tif}")

# Final FALLOW result calculation
def calculate_final_fallow_mask(q1_dir, q2_dir, q3_dir, q4_dir, year, output_fallow_tif, non_fallow_value=1):
    q1_file = os.path.join(q1_dir, f"FALLOW_q1_{year}.tif")
    q2_file = os.path.join(q2_dir, f"FALLOW_q2_{year}.tif")
    q3_file = os.path.join(q3_dir, f"FALLOW_q3_{year}.tif")
    q4_file = os.path.join(q4_dir, f"FALLOW_q4_{year}.tif")
    with rasterio.open(q1_file) as src_q1, \
         rasterio.open(q2_file) as src_q2, \
         rasterio.open(q3_file) as src_q3, \
         rasterio.open(q4_file) as src_q4:
        profile = src_q1.profile
        profile.update(count=1, dtype=rasterio.uint8)
        q1_data = (src_q1.read(1) == 255).astype(np.uint8)
        q2_data = (src_q2.read(1) == 255).astype(np.uint8)
        q3_data = (src_q3.read(1) == 255).astype(np.uint8)
        q4_data = (src_q4.read(1) == 255).astype(np.uint8)
        true_count = q1_data + q2_data + q3_data + q4_data
        final_fallow = np.zeros_like(true_count, dtype=np.uint8)
        final_fallow[true_count >= 2] = 255
        final_fallow[true_count < 2] = non_fallow_value
        with rasterio.open(output_fallow_tif, 'w', **profile) as dst:
            dst.write(final_fallow, 1)
    print(f"Final FALLOW saved: {output_fallow_tif}")

# Function to process FALLOW detection for a specific year
def process_year(region_name, tandvi_dir, tandvirange_dir, ndvi_dir, ndvi_range_dir, median_cd_file, median_cd_range_file, output_fallow_dir, year):
    output_q1 = os.path.join(output_fallow_dir, f"FALLOW_q1_{year}.tif")
    output_q2 = os.path.join(output_fallow_dir, f"FALLOW_q2_{year}.tif")
    output_q3 = os.path.join(output_fallow_dir, f"FALLOW_q3_{year}.tif")
    output_q4 = os.path.join(output_fallow_dir, f"FALLOW_q4_{year}.tif")
    output_final = os.path.join(output_fallow_dir, f"Final_FALLOW_{year}.tif")

    # Call each FALLOW detection function
    calculate_fallow_tandvi(tandvi_dir, year, output_q1)
    calculate_fallow_tandvirange(tandvirange_dir, year, output_q2)
    calculate_fallow_ndvi(ndvi_dir, median_cd_file, year, output_q3)
    calculate_fallow_ndvi_range(ndvi_range_dir, median_cd_range_file, year, output_q4)
    calculate_final_fallow_mask(output_fallow_dir, output_fallow_dir, output_fallow_dir, output_fallow_dir, year, output_final)

# Function to process FALLOW detection for a specific region (parallel processing for years)
def process_region(region_name, base_dir, start_year=2013, end_year=2023):
    region_dir = os.path.join(base_dir, "preprocessed", region_name)
    
    # Set paths
    tandvi_dir = os.path.join(region_dir, "tandvi")
    tandvirange_dir = os.path.join(region_dir, "tandvirange")
    ndvi_dir = os.path.join(region_dir, "monthly_max")
    ndvi_range_dir = os.path.join(region_dir, "range")
    median_cd_file = os.path.join(region_dir, "MedianCD_Monthly.yaml")
    median_cd_range_file = os.path.join(region_dir, "MedianCD_Range_Monthly.yaml")
    output_fallow_dir = os.path.join(region_dir, "fanta")
    
    # Create the fanta folder if it does not exist
    os.makedirs(output_fallow_dir, exist_ok=True)
    
    # Parallel execution for each year
    Parallel(n_jobs=-1)(
        delayed(process_year)(
            region_name, tandvi_dir, tandvirange_dir, ndvi_dir, ndvi_range_dir, median_cd_file, median_cd_range_file, output_fallow_dir, year
        ) for year in tqdm(range(start_year, end_year + 1), desc=f"Processing Years for {region_name}")
    )

# Function to process FALLOW detection for all regions
def process_all_regions(base_dir, start_year=2013, end_year=2023):
    preprocessed_base_dir = os.path.join(base_dir, "preprocessed")
    
    # Iterate over all region directories
    region_names = [name for name in os.listdir(preprocessed_base_dir) if os.path.isdir(os.path.join(preprocessed_base_dir, name))]
    
    for region_name in tqdm(region_names, desc="Processing Regions"):
        process_region(region_name, base_dir, start_year, end_year)

# Execution
base_dir = "/mnt/raid5/1114/"
process_all_regions(base_dir, start_year=2020, end_year=2023)